In [1]:
from torch.utils.data import DataLoader
from torch import Tensor
from torch import optim
from torch.optim.adam import Adam

from torchvision.datasets import CIFAR10 # type: ignore
from torchvision.transforms import Compose, ToTensor, Normalize # type: ignore

import utils

In [ ]:
transform = Compose([ # type: ignore
    ToTensor(),
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader: DataLoader[tuple[Tensor, Tensor]] = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader: DataLoader[tuple[Tensor, Tensor]] = DataLoader(test_dataset, batch_size=64, shuffle=False)

cfgs = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

model = utils.BayesianVGG(cfgs['VGG16'])

optimizer = Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.97)


def reg_coef_lambda(epoch: int) -> float:
    if epoch < 100:
        return 0
    elif epoch < 200:
        return (epoch - 100) / 100
    else:
        return 1

In [ ]:
trainer = utils.Trainer(model, train_loader, len(train_dataset), test_loader, len(test_dataset), optimizer, scheduler, epochs=500)
trainer.train()